In this notebook we create few plots of wind intensity measured by satellite, with a particular interest on the Canary Islands, which frequently generate the so-called *von Karman* vortex street.

In [3]:
import datetime
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import windplot
import datareading
import logging
import copy
from importlib import reload
reload(windplot)

<module 'windplot' from '/home/ctroupin/Projects/FNRS-UpwellingFilaments/Filament-Submesoscale/python/windplot.py'>

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("Starting")

INFO:root:Starting


## Input parameters
We set the date of interest and the domain coordinates are taken from the geoTIFF file.

In [23]:
year, month, day = 2019, 11, 28
figdir = "../figures/eddy/"
visibledir = "/data/Visible/Balearic/"
os.path.exists(figdir)

True

## Data

In [11]:
datewind = datetime.datetime(year, month, day)
dayofyear = datewind.timetuple().tm_yday

### Visible image

In [18]:
visibleim = datewind.strftime("MODIS-Aqua-%Y-%m-%dT00_00_00Z.tiff")
#visibleim = datewind.strftime("Sentinel-3_OLCI_%Y-%m-%d.tiff")
#visibleim = datewind.strftime("Sentinel-2_L1C_%Y-%m-%d.tiff")
#visibleim = datewind.strftime("Terra-%Y-%m-%dT00_00_00Z.tiff")
if os.path.exists(os.path.join(visibledir, visibleim)):
    logger.info("Reading geoTIFF file {}".format(visibleim))
    lonvis, latvis, arr, inproj, extent = datareading.read_geotiff(os.path.join(visibledir, visibleim))
    extent2 = copy.deepcopy(extent)
    extent2[0] -= 360.
    extent2[1] -= 360.
else:
    logger.info("No visible image for that date")

INFO:root:Reading geoTIFF file MODIS-Aqua-2019-11-28T00_00_00Z.tiff


### Scatterometer wind
The wind (netCDF) are read from the JPL OPEnDAP server.      
The files are sorted by years and days of year.

With these pieces of information we can create two lists of files (one for MetOp-A, one for MetOp-A).

In [26]:
filelist1 = windplot.get_filelist_url(year, dayofyear)
reload(windplot)
filelist1 = ["https://opendap.jpl.nasa.gov:443/opendap/OceanWinds/ascat/preview/L2/metop_b/coastal_opt/2019/332/ascat_20191128_092400_metopb_37332_eps_o_coa_3202_ovw.l2.nc.gz"]

INFO:windplot:Found 28 files
INFO:root:Starting


In [27]:
for ifile, dataurl in enumerate(filelist1):

    logger.info("Working on {}".format(dataurl))
    lon, lat, uwind, vwind, windspeed = windplot.read_data_domain(dataurl, extent2)

    if lon is not None:    
        mm = str(month).zfill(2)
        dd = str(day).zfill(2)
        
        fname = "".join(("Aqua-", dataurl.split("/")[-1].split(".")[0]))
        logger.info("Saving figure as {}".format(fname))
        figname = os.path.join(figdir, fname)
        windplot.plot_wind_sat(lon, lat, uwind, vwind, windspeed, arr, extent, figname,
                               cmap=plt.cm.hot_r, clim=[0, 15], 
                               date="{}-{}-{}".format(year, mm, dd))

INFO:root:Working on https://opendap.jpl.nasa.gov:443/opendap/OceanWinds/ascat/preview/L2/metop_b/coastal_opt/2019/332/ascat_20191128_092400_metopb_37332_eps_o_coa_3202_ovw.l2.nc.gz
INFO:windplot:Subsetting data to region of interest
INFO:root:Saving figure as Aqua-ascat_20191128_092400_metopb_37332_eps_o_coa_3202_ovw
